In [18]:
import pytesseract
from PIL import Image
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

im=Image.open('test.jpg')
# print(pytesseract.image_to_string(im))

im = im.convert('L')  # 转为灰度图

# 二值化
def iterGetThreshold(img, pixdata, width, height):
    """
    迭代法，求阈值
    """
    pixPrs = pixBac = []  # 用于统计前景和背景平均阈值
    threshold = 0
    pixel_min, pixel_max = img.getextrema()  # 获得图片中最大和最小灰度值
    newThreshold = int((pixel_min + pixel_max) / 2)  # 初始阈值

    while True:
        if abs(threshold - newThreshold) < 5:  # 差值小于5,退出
            break
        for y in range(height):
            for x in range(width):
                if pixdata[x, y] >= newThreshold:
                    pixBac.append(pixdata[x, y])  # 大于阈值 为背景
                else:
                    pixPrs.append(pixdata[x, y])  # 小于， 前景

        avgPrs = sum(pixPrs) / len(pixPrs)
        avgBac = sum(pixBac) / len(pixBac)
        threshold = newThreshold
        newThreshold = int((avgPrs + avgBac) / 2)

    return newThreshold


def binary(img, threshold=None):
    """
    二值化
    """
    img = img.convert('L')  # 转为灰度图
    pixdata = img.load()
    width, height = img.size

    if not threshold:
        threshold = iterGetThreshold(img, pixdata, width, height)
    # 遍历所有像素，大于阈值的为白色
    for y in range(height):
        for x in range(width):
            if pixdata[x, y] < threshold:
                pixdata[x, y] = 0
            else:
                pixdata[x, y] = 255

    return img

new_img = binary(im)

def depoint(img, N=2):
    """
    降噪
    """
    pixdata = img.load()
    width, height = img.size
    for y in range(1, height - 1):
        for x in range(1, width - 1):
            count = 0
            if pixdata[x, y - 1] == 255:  # 上
                count = count + 1
            if pixdata[x, y + 1] == 255:  # 下
                count = count + 1
            if pixdata[x - 1, y] == 255:  # 左
                count = count + 1
            if pixdata[x + 1, y] == 255:  # 右
                count = count + 1
            if pixdata[x-1, y-1] == 255:  #左上
                count = count + 1
            if pixdata[x+1, y-1] == 255:  #右上
                count = count + 1
            if pixdata[x-1, y+1] == 255:  #左下
                count = count + 1
            if pixdata[x+1, y+1] == 255:  #右下
                count = count + 1

            if count > N:
                pixdata[x, y] = 255  # 设置为白色
    return img

img = depoint(new_img, 4)

print(pytesseract.image_to_string(img))


 

8502



In [30]:
# # 案例模拟登录古诗文网 https://so.gushiwen.cn/
#
# import requests
# import pytesseract
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
#
# from lxml import etree
# from io import BytesIO
# from PIL import Image
#
# url = "https://so.gushiwen.cn/user/login.aspx?from=http://so.gushiwen.cn/user/collect.aspx"
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36",
#     "cookie": "login=flase; ASP.NET_SessionId=ms3045g3ed3aksg5up5z2bgp; Hm_lvt_9007fab6814e892d3020a64454da5a55=1615100808,1615111067; login=flase; gswZhanghao=2321936402%40qq.com; wsEmail=2321936402%40qq.com; Hm_lpvt_9007fab6814e892d3020a64454da5a55=1615111730; codeyzgswso=8a8a9933b1cd4bff"
# }
# response = requests.get(url, headers=headers)
#
# tree = etree.HTML(response.text)
# img_src = "https://so.gushiwen.cn/" + tree.xpath('//*[@id="imgCode"]/@src')[0]
#
# img_data = requests.get(img_src, headers=headers).content
# img_io = BytesIO(img_data)
#
# im = Image.open(img_io)
# w, h = im.size
# im = im.resize((w*4, h*4))
#
# im = im.convert('L')
#
# def iterGetThreshold(img, pixdata, width, height):
#     """
#     迭代法，求阈值
#     """
#     pixPrs = pixBac = []  # 用于统计前景和背景平均阈值
#     threshold = 0
#     pixel_min, pixel_max = img.getextrema()  # 获得图片中最大和最小灰度值
#     newThreshold = int((pixel_min + pixel_max) / 2)  # 初始阈值
#
#     while True:
#         if abs(threshold - newThreshold) < 5:  # 差值小于5,退出
#             break
#         for y in range(height):
#             for x in range(width):
#                 if pixdata[x, y] >= newThreshold:
#                     pixBac.append(pixdata[x, y])  # 大于阈值 为背景
#                 else:
#                     pixPrs.append(pixdata[x, y])  # 小于， 前景
#
#         avgPrs = sum(pixPrs) / len(pixPrs)
#         avgBac = sum(pixBac) / len(pixBac)
#         threshold = newThreshold
#         newThreshold = int((avgPrs + avgBac) / 2)
#
#     return newThreshold
#
#
# def binary(img, threshold=None):
#     """
#     二值化
#     """
#     img = img.convert('L')  # 转为灰度图
#     pixdata = img.load()
#     width, height = img.size
#
#     if not threshold:
#         threshold = iterGetThreshold(img, pixdata, width, height)
#     # 遍历所有像素，大于阈值的为白色
#     for y in range(height):
#         for x in range(width):
#             if pixdata[x, y] < threshold:
#                 pixdata[x, y] = 0
#             else:
#                 pixdata[x, y] = 255
#
#     return img
#
# im = binary(im)
#
# def depoint(img, N=4):
#     """
#     降噪
#     """
#     pixdata = img.load()
#     width, height = img.size
#     for y in range(1, height - 1):
#         for x in range(1, width - 1):
#             count = 0
#             if pixdata[x, y - 1] == 255:  # 上
#                 count = count + 1
#             if pixdata[x, y + 1] == 255:  # 下
#                 count = count + 1
#             if pixdata[x - 1, y] == 255:  # 左
#                 count = count + 1
#             if pixdata[x + 1, y] == 255:  # 右
#                 count = count + 1
#             if pixdata[x-1, y-1] == 255:  #左上
#                 count = count + 1
#             if pixdata[x+1, y-1] == 255:  #右上
#                 count = count + 1
#             # if pixdata[x-1, y+1] == 255:  #左下
#             #     count = count + 1
#             # if pixdata[x+1, y+1] == 255:  #右下
#             #     count = count + 1
#
#             if count > N:
#                 pixdata[x, y] = 255  # 设置为白色
#     return img
#
# im = depoint(im)
# im.show()
#
# # print(pytesseract.image_to_string(im))

# 超级鹰接入

In [50]:
from chaojiying import Chaojiying_Client

chaojiying = Chaojiying_Client('dyp1996', 'DYP-abcd-1996', '913681')


import requests
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

from lxml import etree
from io import BytesIO
from PIL import Image

session = requests.Session()

url = "https://so.gushiwen.cn/user/login.aspx?from=http://so.gushiwen.cn/user/collect.aspx"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36",
}
response = session.get(url, headers=headers)

tree = etree.HTML(response.text)
img_src = "https://so.gushiwen.cn/" + tree.xpath('//*[@id="imgCode"]/@src')[0]

__VIEWSTATE = tree.xpath("//input[@id='__VIEWSTATE']/@value")[0]
# print(hidden)
__VIEWSTATEGENERATOR = tree.xpath("//input[@id='__VIEWSTATEGENERATOR']/@value")[0]
print(__VIEWSTATEGENERATOR, __VIEWSTATE)

img_data = session.get(img_src, headers=headers).content
img_io = BytesIO(img_data)

result = chaojiying.PostPic(img_io, 1902)

code = result.get("pic_str")

print(code)
Image.open(img_io).show()

login_url = "https://so.gushiwen.cn/user/login.aspx?from=http%3a%2f%2fso.gushiwen.cn%2fuser%2fcollect.aspx"

data = {
    "__VIEWSTATE": __VIEWSTATE,
    "__VIEWSTATEGENERATOR": __VIEWSTATEGENERATOR,
    "from": "http://so.gushiwen.cn/user/collect.aspx",
    "email": "2321936402@qq.com",
    "pwd": "abcd1996",
    "code": code,
    "denglu": "登录"
}

resp = session.post(login_url, headers=headers, data=data)
with open('login.html', 'w', encoding='utf-8') as f:
    f.write(resp.text)


C93BE1AE xiPwJpg/3rhxun0KTCL8an6shBdzf5QXB2vjGSYrwLeG7Vh6n68QUaLX011YduUlw38LApt1DUBD7ghedadqRvxVA1Y1c4VsHK628zhOQc2E8+O+io7+hOm1GTs=
jxhn
